In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('../..')
import glb
from dgl.nn import GraphConv

metadata_path = "../../examples/citeseer/metadata.json"
task_path = "../../examples/citeseer/task.json"
g = glb.graph.read_glb_graph(metadata_path=metadata_path)
task = glb.task.read_glb_task(task_path=task_path)

dataset = glb.dataloading.combine_graph_and_task(g, task)
g = dataset[0]

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['NodeFeature'].shape[1], 16, dataset._num_labels)

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['NodeFeature']
    labels = g.ndata['NodeLabel']
    train_mask = g.ndata['train_set']
    val_mask = g.ndata['val_set']
    test_mask = g.ndata['test_set']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

# train with cpu
model = GCN(g.ndata['NodeFeature'].shape[1], 16, dataset._num_labels)
train(g, model)

# train with gpu
# g = g.to('cuda')
# model = GCN(g.ndata['NodeFeature'].shape[1], 16, dataset._num_labels).to('cuda')
# train(g, model)

CITESEER dataset.
Node classification on CITESEER dataset. Planetoid split.
In epoch 0, loss: 1.792, val acc: 0.198 (best 0.198), test acc: 0.194 (best 0.194)
In epoch 5, loss: 1.745, val acc: 0.440 (best 0.456), test acc: 0.413 (best 0.425)
In epoch 10, loss: 1.663, val acc: 0.566 (best 0.566), test acc: 0.537 (best 0.537)
In epoch 15, loss: 1.550, val acc: 0.592 (best 0.600), test acc: 0.575 (best 0.585)
In epoch 20, loss: 1.409, val acc: 0.610 (best 0.610), test acc: 0.594 (best 0.594)
In epoch 25, loss: 1.244, val acc: 0.626 (best 0.630), test acc: 0.605 (best 0.601)
In epoch 30, loss: 1.064, val acc: 0.628 (best 0.632), test acc: 0.606 (best 0.609)
In epoch 35, loss: 0.881, val acc: 0.636 (best 0.636), test acc: 0.614 (best 0.612)
In epoch 40, loss: 0.710, val acc: 0.646 (best 0.648), test acc: 0.616 (best 0.618)
In epoch 45, loss: 0.561, val acc: 0.656 (best 0.656), test acc: 0.619 (best 0.619)
In epoch 50, loss: 0.439, val acc: 0.658 (best 0.658), test acc: 0.622 (best 0.618)
In